### Single Chromosome Simulation for Classic MiChroM

This tutorial should take between 10 to 20 minutes of reading and performing simulations.

#### Chromatin Dynamics Simulations on Chromosome 10 of stomach GRCh38 (from ENCODE Project)

The first step is to import the **OpenMiChroM** module

In [2]:
from OpenMiChroM.ChromDynamics import MiChroM

Download the bed file that contains the sequence annotation for stomach via ENCODE website (https://www.encodeproject.org/)

In [ ]:
%%bash
wget -O Tutorials/Chromosomes_simulations/inputs/ENCFF963ZUJ.bed.gz https://www.encodeproject.org/files/ENCFF963ZUJ/@@download/ENCFF963ZUJ.bed.gz 
gunzip -f Tutorials/Chromosomes_simulations/inputs/ENCFF963ZUJ.bed.gz 


`MiChroM` class sets the initial parameters of the simulation:

- `timeStep=0.01`: set the simulation time step to perfom the integration<br>
- `temperature=1.0`: set the temperature of your simulation<br>

In [16]:
sim = MiChroM(name='stomach_GRCh38', temperature=1.0, timeStep=0.01)

    ***************************************************************************************     
     **** **** *** *** *** *** *** *** OpenMiChroM-1.1.0 *** *** *** *** *** *** **** ****      

         OpenMiChroM is a Python library for performing chromatin dynamics simulations.         
                            OpenMiChroM uses the OpenMM Python API,                             
                employing the MiChroM (Minimal Chromatin Model) energy function.                
      The chromatin dynamics simulations generate an ensemble of 3D chromosomal structures      
      that are consistent with experimental Hi-C maps, also allows simulations of a single      
                 or multiple chromosome chain using High-Performance Computing                  
                            in different platforms (GPUs and CPUs).                             

         OpenMiChroM documentation is available at https://open-michrom.readthedocs.io          

         OpenMiChroM is des

There are four hardware platform options to run the simulations: 
```python
platform="cuda"
platform="opencl"
platform="hip"
platform="cpu"
```

Choose accordingly.

In [19]:
sim.setup(platform="Cuda")

Using platform: CUDA


Set the directory name in which the output of the simulation is saved:

In [18]:
sim.saveFolder('/home/antonio/test_OM1.1')

The next step is to load the chromatin compartment sequence for chromosome 10 and generate an initial 3D structure to start the simulation. We can use the [createSpringSpiral](https://open-michrom.readthedocs.io/en/latest/OpenMiChroM.html#OpenMiChroM.ChromDynamics.MiChroM.createSpringSpiral) function to set the initial configuration of the polymer based in the sequence file.

We will use the bed file download above, and set the chromosome 10 to slice the file and get the sequence annotation.

In [20]:
sim.buildClassicMichrom(ChromSeq='Tutorials/Chromosomes_simulations/inputs/ENCFF963ZUJ.bed', chromosome='chr10')


FENEBond was added
AngleForce was added
RepulsiveSoftCore was added
TypetoType was added
IdealChromosome was added
FlatBottomHarmonic was added
Setting positions... loaded!
Setting velocities... loaded!
Context created!

Simulation name: stomach_GRCh38
Number of beads: 2676, Number of chains: 1
Potential energy: 63.01043, Kinetic Energy: 1.51421 at temperature: 1.0

Potential energy per forceGroup:
                                  Values
FENEBond                   54584.082069
AngleForce                     1.002699
RepulsiveSoftCore              0.000000
TypetoType                  -212.000758
IdealChromosome               -1.071779
FlatBottomHarmonic        114243.908772
Potential Energy (total)  168615.921003


As you can see on the output above, we build the system with MiChroM Potential add the homopolymer potentials and the Michrom Potentials.

The system reports some statitics as, number of beads, number of chains and the initial energy potential for each force applied.

Now we create the reporters to save the simulation infos. There are 3 types of reporters:

**statistics**: Attaches a reporter to collect simulation statistics such as step number, radius of gyration (RG), total energy, potential energy, kinetic energy, and temperature.

**trajectory**:  Attaches a reporter to save trajectory data (xyz per bead per chain) during the simulation. The file format to save the trajectory data. Options are 'cndb', 'swb','ndb', 'pdb', 'gro', 'xyz'. (Default: 'cndb')

**energy components**: Saves energy components per force group to a separate file named 'energyComponents.txt' in the simulation folder. Requires that statistics is True

set the number of steps interval we will save this information, here I choose 1000 steps

In [21]:
sim.createReporters(statistics=True, traj=True, trajFormat="cndb", energyComponents=True, interval=10**3)


The `sim.run()` function is used to start the simulation. The parameters for this function are:

- `nsteps`: The number of steps to run the simulation. In this case, it is set to \(10^5\).
- `report`: A boolean value indicating whether to report the simulation progress. Here, it is set to `True`.
- `interval`: The interval at which the simulation reports progress. In this case, it is set to \(10^4\) steps.



In [ ]:
sim.run(nsteps=10**6, report=True, interval=10**4)

After the simulation end, in the output folder you wil find the files, in this tutorial the files was saved in "/home/antonio/test_OM1.1"

In [27]:
%%bash
cat /home/antonio/test_OM1.1/initialStats.txt

Platform Name: CUDA
Platform Speed: 100.0
Platform Property Names: ('DeviceIndex', 'DeviceName', 'UseBlockingSync', 'Precision', 'UseCpuPme', 'CudaCompiler', 'TempDirectory', 'CudaHostCompiler', 'DisablePmeStream', 'DeterministicForces')
DeviceIndex Value: 0
DeviceName Value: NVIDIA GeForce RTX 3070 Laptop GPU
UseBlockingSync Value: false
Precision Value: mixed
UseCpuPme Value: false
CudaCompiler Value: 
TempDirectory Value: /tmp
CudaHostCompiler Value: 
DisablePmeStream Value: false
DeterministicForces Value: false

Simulation name: stomach_GRCh38
Number of beads: 2676, Number of chains: 1
Potential energy: 63.01043, Kinetic Energy: 1.51421 at temperature: 1.0

Potential energy per forceGroup:
                                  Values
FENEBond                   54584.082069
AngleForce                     1.002699
RepulsiveSoftCore              0.000000
TypetoType                  -212.000758
IdealChromosome               -1.071779
FlatBottomHarmonic        114243.908772
Potential Energ

In [28]:
%%bash
head -n 10 /home/antonio/test_OM1.1/statistics.txt

#Step,RG,Etotal,Epot,Ekin,Temperature

1000,142.264,64.100,61.616,2.483,1.656
2000,136.801,61.388,58.304,3.084,2.056
3000,130.833,58.047,54.864,3.183,2.122
4000,124.858,54.758,51.551,3.207,2.138
5000,119.155,51.522,48.544,2.978,1.986
6000,113.643,48.609,45.721,2.888,1.925
7000,108.429,45.973,43.181,2.792,1.861
8000,103.459,43.588,40.932,2.656,1.771


In [29]:
%%bash
head -n 10 /home/antonio/test_OM1.1/energyComponents.txt

['Step', 'FENEBond', 'AngleForce', 'RepulsiveSoftCore', 'TypetoType', 'IdealChromosome', 'FlatBottomHarmonic']
1000,55437.266,533.813,0.000,-255.140,-2.148,109170.855
2000,55337.437,1032.451,1.045,-328.348,-9.719,99987.543
3000,55395.351,1398.317,2.719,-380.766,-32.588,90433.013
4000,55472.516,1613.201,2.975,-428.613,-66.608,81357.142
5000,55455.042,1892.398,14.935,-486.590,-115.180,73142.277
6000,55439.545,1985.718,20.307,-541.609,-176.084,65621.029
7000,55338.329,2176.605,25.564,-622.169,-247.391,58881.970
8000,55452.549,2249.492,55.201,-702.398,-321.593,52800.003
9000,55405.277,2350.506,47.454,-766.710,-377.838,47393.514
